# An attempt at making some functions for feature generation

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('data/trades_AAPL_20230106-20230110.csv.gz', compression='gzip')

data['Time']=pd.to_datetime(data['Time'])

print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   Unnamed: 0                              100000 non-null  int64         
 1   Time                                    100000 non-null  datetime64[ns]
 2   Date                                    100000 non-null  object        
 3   Exchange                                100000 non-null  object        
 4   Symbol                                  100000 non-null  object        
 5   Trade_Volume                            100000 non-null  int64         
 6   Trade_Price                             100000 non-null  float64       
 7   Sale_Condition                          100000 non-null  object        
 8   Source_of_Trade                         100000 non-null  object        
 9   Trade_Stop_Stock_Indicator            

In [3]:
selected_columns = ["Time", "Date", "Symbol", "Trade_Price", "Trade_Volume", "YearMonth"]
filtered_data = data[selected_columns]

print(filtered_data.info())
print(filtered_data.head())
print(filtered_data.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Time          100000 non-null  datetime64[ns]
 1   Date          100000 non-null  object        
 2   Symbol        100000 non-null  object        
 3   Trade_Price   100000 non-null  float64       
 4   Trade_Volume  100000 non-null  int64         
 5   YearMonth     100000 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.6+ MB
None
                        Time        Date Symbol  Trade_Price  Trade_Volume  \
0 2023-01-06 04:00:00.007354  2023-01-06   AAPL       125.24            40   
1 2023-01-06 04:00:00.009587  2023-01-06   AAPL       125.22             1   
2 2023-01-06 04:00:00.014786  2023-01-06   AAPL       125.22             4   
3 2023-01-06 04:00:00.016263  2023-01-06   AAPL       125.22             1   
4 2023-

In [ ]:
# Assuming 'filtered_data' is your DataFrame with the raw high-frequency trading data.

# Function to find and return duplicate DateTime entries in the data
def find_duplicates(df):
    """
    Identify and return a DataFrame containing duplicate entries based on DateTime.

    :param df: DataFrame with high-frequency trading data.
    :return: DataFrame with duplicate DateTime entries.
    """
    # Convert 'Time' and 'Date' to a single datetime column
    # df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    
    # Check for duplicates in the DateTime column
    duplicate_rows = df[df.duplicated('Time', keep=False)]
    
    return duplicate_rows

# Call the function to find duplicates
duplicate_entries = find_duplicates(filtered_data)

print(duplicate_entries.head())

In [4]:
# Assuming 'filtered_data' is your DataFrame with the raw high-frequency trading data.

# Function to resample DataFrame to 1-minute intervals
def resample_data(df, resample_rate='1T', forward_fill=True):
    """
    Resample the time series data to a specified rate.

    :param df: DataFrame with high-frequency trading data.
    :param resample_rate: The frequency rate to resample the DataFrame. Default is '1T' (1 minute).
    :return: Resampled DataFrame with the original columns.
    """
    # Make a copy of the DataFrame to avoid SettingWithCopyWarning
    df_copy = df.copy()
    
    # Convert 'Time' and 'Date' to a single datetime column and set as index
    df_copy.set_index('Time', inplace=True)
    
    # Sort the index just in case it's not sorted
    df_copy.sort_index(inplace=True)
    
    # Drop duplicate indices
    df_copy = df_copy[~df_copy.index.duplicated(keep='first')]
    
    # Define the aggregation dictionary for resampling
    aggregation = {
        'Trade_Price': 'last',
        'Trade_Volume': 'sum'  # Assuming you want to sum the volumes within the interval
    }
    
    # Resample the DataFrame
    resampled_df = df_copy.resample(resample_rate).agg(aggregation)
    
    # Forward fill the Trade_Price to handle NaN values if there are no trades in the interval
    if forward_fill: resampled_df['Trade_Price'].ffill(inplace=True)
    
    # Merge the resampled data back with the original dataframe to preserve non-numeric columns
    # This aligns the non-numeric data with the resampled numeric data
    df_non_numeric = df_copy[['Date', 'Symbol', 'YearMonth']].resample(resample_rate).ffill()
    full_resampled_df = pd.concat([df_non_numeric, resampled_df], axis=1)
    
    return full_resampled_df

# Call the function with the dataframe and the resampling rate
# Example: resampled_dataframe = resample_data(filtered_data, '1T')


In [5]:
resampled_dataframe = resample_data(filtered_data, '1T')
print(resampled_dataframe.info())
print(resampled_dataframe.head())
print(resampled_dataframe.tail())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5040 entries, 2023-01-06 04:00:00 to 2023-01-09 15:59:00
Freq: T
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          5039 non-null   object 
 1   Symbol        5039 non-null   object 
 2   YearMonth     5039 non-null   float64
 3   Trade_Price   5040 non-null   float64
 4   Trade_Volume  5040 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 236.2+ KB
None
                           Date Symbol  YearMonth  Trade_Price  Trade_Volume
Time                                                                        
2023-01-06 04:00:00         NaN    NaN        NaN       125.34          3262
2023-01-06 04:01:00  2023-01-06   AAPL   202301.0       125.33          3816
2023-01-06 04:02:00  2023-01-06   AAPL   202301.0       125.21          2461
2023-01-06 04:03:00  2023-01-06   AAPL   202301.0       125.23          1367
2023-01-06 04:04:0

In [ ]:
VOL_LOOKBACK = 60  # for ex-ante volatility (in minutes)
VOL_TARGET = 0.15  # 15% volatility target

def calc_returns(srs: pd.Series, minute_offset: int = 1) -> pd.Series:
    """for each element of a pandas time-series srs,
    calculates the returns over the past number of minutes
    specified by offset

    Args:
        srs (pd.Series): time-series of prices
        minute_offset (int, optional): number of minutes to calculate returns over. Defaults to 1.

    Returns:
        pd.Series: series of returns
    """
    returns = srs / srs.shift(minute_offset) - 1.0
    return returns

def calc_minute_vol(minute_returns, lookback_minutes=60):
    return (
        minute_returns.ewm(span=lookback_minutes, min_periods=lookback_minutes)
        .std()
        .fillna(method="bfill")
    )

def calc_normalised_returns(minute_offset, df_asset):
        return (
            calc_returns(df_asset["srs"], minute_offset)
            / df_asset["daily_vol"]
            / np.sqrt(minute_offset)
        )

# def calc_vol_scaled_returns(daily_returns, daily_vol=pd.Series(None)):
#     """calculates volatility scaled returns for annualised VOL_TARGET of 15%
#     with input of pandas series daily_returns"""
#     if not len(daily_vol):
#         daily_vol = calc_minute_vol(daily_returns)
#     annualised_vol = daily_vol * np.sqrt(252)  # annualised
#     return daily_returns * VOL_TARGET / annualised_vol.shift(1)

class MACDStrategy:
    def __init__(self, trend_combinations: List[Tuple[float, float]] = None):
        """Used to calculated the combined MACD signal for a multiple short/signal combinations,
        as described in https://arxiv.org/pdf/1904.04912.pdf

        Args:
            trend_combinations (List[Tuple[float, float]], optional): short/long trend combinations. Defaults to None.
        """
        if trend_combinations is None:
            self.trend_combinations = [(8, 24), (16, 48), (32, 96)]
        else:
            self.trend_combinations = trend_combinations

    @staticmethod
    def calc_signal(srs: pd.Series, short_timescale: int, long_timescale: int) -> float:
        """Calculate MACD signal for a signal short/long timescale combination

        Args:
            srs ([type]): series of prices
            short_timescale ([type]): short timescale
            long_timescale ([type]): long timescale

        Returns:
            float: MACD signal
        """

        def _calc_halflife(timescale):
            return np.log(0.5) / np.log(1 - 1 / timescale)

        macd = (
            srs.ewm(halflife=_calc_halflife(short_timescale)).mean()
            - srs.ewm(halflife=_calc_halflife(long_timescale)).mean()
        )
        q = macd / srs.rolling(63).std().fillna(method="bfill")
        return q / q.rolling(252).std().fillna(method="bfill")

    @staticmethod
    def scale_signal(y):
        return y * np.exp(-(y ** 2) / 4) / 0.89

    def calc_combined_signal(self, srs: pd.Series) -> float:
        """Combined MACD signal

        Args:
            srs (pd.Series): series of prices

        Returns:
            float: MACD combined signal
        """
        return np.sum(
            [self.calc_signal(srs, S, L) for S, L in self.trend_combinations]
        ) / len(self.trend_combinations)


In [ ]:
VOL_THRESHOLD = 5  # multiple to winsorise by
HALFLIFE_WINSORISE = 390 #number of minutes in a trading day. Not sure if this is an appropriate value or what

# winsorize using rolling 5X standard deviations to remove outliers
resampled_dataframe["srs"] = resampled_dataframe["Trade_Price"]
ewm = resampled_dataframe["srs"].ewm(halflife=HALFLIFE_WINSORISE)
means = ewm.mean()
stds = ewm.std()
resampled_dataframe["srs"] = np.minimum(resampled_dataframe["srs"], means + VOL_THRESHOLD * stds)
resampled_dataframe["srs"] = np.maximum(resampled_dataframe["srs"], means - VOL_THRESHOLD * stds)

resampled_dataframe["minute_returns"] = calc_returns(resampled_dataframe["srs"])
resampled_dataframe["minute_vol"] = calc_minute_vol(resampled_dataframe["minute_returns"])

# vol scaling and shift to be next day returns
# resampled_dataframe["target_returns"] = calc_vol_scaled_returns(
#     resampled_dataframe["daily_returns"], resampled_dataframe["daily_vol"]
# ).shift(-1)

def calc_normalised_returns(minute_offset):
    return (
        calc_returns(resampled_dataframe["srs"], minute_offset)
        / resampled_dataframe["minute_vol"]
        / np.sqrt(minute_offset)
    )

# intervals_in_minutes = [20, 60, 120, 180, 240]  # 20 minutes, 1 hour, 2 hours, 3 hours, 4 hours

resampled_dataframe["norm_min_return"] = calc_normalised_returns(1)
resampled_dataframe["norm_20min_return"] = calc_normalised_returns(20)
resampled_dataframe["norm_60min_return"] = calc_normalised_returns(60)
resampled_dataframe["norm_120min_return"] = calc_normalised_returns(120)
resampled_dataframe["norm_180min_return"] = calc_normalised_returns(180)

trend_combinations = [(8, 24), (16, 48), (32, 96)]
for short_window, long_window in trend_combinations:
    resampled_dataframe[f"macd_{short_window}_{long_window}"] = MACDStrategy.calc_signal(
        resampled_dataframe["srs"], short_window, long_window
    )

In [ ]:
print(resampled_dataframe.info())
print(resampled_dataframe.head())
print(resampled_dataframe.tail())

In [ ]:
# Plot the Trade_Price and the winsorized srs series on the same plot. Trying to see differnece

plt.figure(figsize=(14, 7))

# Plot original Trade_Price
plt.plot(resampled_dataframe.index, resampled_dataframe["Trade_Price"], label='Trade Price', color='blue')

# Plot winsorized srs
plt.plot(resampled_dataframe.index, resampled_dataframe["srs"], label='Winsorized SRS', color='orange')

plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Trade Price vs Winsorized SRS')
plt.legend()
plt.show()

In [ ]:
# Modified function to calculate multiple MACD indicators based on arrays of window parameters

def calculate_multiple_macds(df, short_windows, long_windows, signal_windows):
    """
    Calculate multiple MACD indicators for each time step based on arrays of window sizes.

    :param df: DataFrame with high-frequency trading data and a datetime index
    :param short_windows: Array of short-term EMA window sizes
    :param long_windows: Array of long-term EMA window sizes
    :param signal_windows: Array of signal line EMA window sizes
    :return: DataFrame with multiple MACD and Signal lines
    """
    # Initialize a DataFrame to store MACD values for each set of windows
    macd_df = pd.DataFrame(index=df.index)
    
    # Loop through all provided window sizes
    for short_window, long_window, signal_window in zip(short_windows, long_windows, signal_windows):
        # Resample the trade prices to 1-minute intervals using the last price in the minute
        price = df['Trade_Price'].resample('1T').last().ffill()
        
        # Calculate the short-term and long-term EMAs of the trade prices
        short_ema = price.ewm(span=short_window, min_periods=1, adjust=False).mean()
        long_ema = price.ewm(span=long_window, min_periods=1, adjust=False).mean()
        
        # Calculate the MACD line and the Signal line
        macd = short_ema - long_ema
        # signal_line = macd.ewm(span=signal_window, min_periods=1, adjust=False).mean()
        
        # Store the MACD and Signal line values in the DataFrame using the window sizes as column names
        macd_df[f'MACD_{short_window}_{long_window}'] = macd
        # macd_df[f'Signal_{short_window}_{long_window}_{signal_window}'] = signal_line

    return macd_df

# Example usage:
# short_windows = [8, 16, 32]
# long_windows = [24, 28, 96]
# signal_windows = [9, 9, 9]  # Assuming the signal line uses a 9-period EMA for all
# macd_values = calculate_multiple_macds(filtered_data, short_windows, long_windows, signal_windows)


In [ ]:
short_windows = [8, 16, 32]
long_windows = [24, 28, 96]
signal_windows = [9, 9, 9]  # Assuming the signal line uses a 9-period EMA for all
macd_values = calculate_multiple_macds(resampled_dataframe, short_windows, long_windows, signal_windows)

print(macd_values.info())
print(macd_values.head())



In [ ]:
# Function to calculate volatility-normalized returns for different time intervals
def calculate_vol_normalized_returns(df, intervals):
    """
    Calculate the volatility-normalized returns for different time intervals.

    :param df: DataFrame with high-frequency trading data and a datetime index.
    :param intervals: List of intervals in minutes for which to calculate the normalized returns.
    :return: DataFrame with volatility-normalized returns for each interval.
    """
    # Calculate log returns
    df['Log_Returns'] = np.log(df['Trade_Price'] / df['Trade_Price'].shift(1))
    
    # Create a DataFrame to store normalized returns for each interval
    normalized_returns_df = pd.DataFrame(index=df.index)
    
    # Loop over the intervals
    for interval in intervals:
        # Calculate the rolling standard deviation (volatility) for the current interval
        rolling_volatility = df['Log_Returns'].rolling(window=interval).std()
        
        # Volatility-normalize the log returns by dividing by the rolling volatility
        # Note that the first 'interval - 1' values will be NaN due to the rolling calculation
        normalized_returns = df['Log_Returns'] / rolling_volatility
        
        # Store the normalized returns in the DataFrame
        normalized_returns_df[f'Normalized_Returns_{interval}min'] = normalized_returns
    
    # Drop the initial rows where the rolling volatility can't be calculated
    normalized_returns_df.dropna(how='all', inplace=True)
    
    return normalized_returns_df

# Example intervals in minutes that might correspond to different intraday time horizons
# Since 1 minute is our smallest increment, I dont think it possible for the volatility normaled returns to be determined since std = 0
# intervals_in_minutes = [1, 60, 180, 360, 720]  # 1 minute, 1 hour, 3 hours, 6 hours, 12 hours

# Assuming 'filtered_data' has a datetime index and a 'Trade_Price' column
# Calculate the volatility-normalized returns
# vol_normalized_returns = calculate_vol_normalized_returns(filtered_data, intervals_in_minutes)
# print(vol_normalized_returns.head())

# Note: This code assumes that 'filtered_data' is already sorted by datetime and has no duplicate indices.


In [ ]:
# Example intervals in minutes that might correspond to different intraday time horizons
intervals_in_minutes = [20, 60, 120, 180, 240]  # 20 minutes, 1 hour, 2 hours, 3 hours, 4 hours

# Assuming 'filtered_data' has a datetime index and a 'Trade_Price' column
# Calculate the volatility-normalized returns
vol_normalized_returns = calculate_vol_normalized_returns(resampled_dataframe, intervals_in_minutes)
print(vol_normalized_returns.info())
print(vol_normalized_returns.head(15))

# Note: This code assumes that 'filtered_data' is already sorted by datetime and has no duplicate indices.


In [ ]:
# Function to calculate draw-ups and draw-downs at different frequencies
def calculate_drawdowns_drawups(df, intervals):
    """
    Calculate draw-ups and draw-downs based on highest and lowest prices over different past intervals.

    :param df: DataFrame with high-frequency trading data and a datetime index.
    :param intervals: List of intervals in minutes for which to calculate draw-ups and draw-downs.
    :return: DataFrame with draw-ups and draw-downs for each interval.
    """
    # Calculate log prices
    df['Log_Price'] = np.log(df['Trade_Price'])
    print(df['Log_Price'].head())
    # Create a DataFrame to store drawdowns and draw-ups for each interval
    features_df = pd.DataFrame(index=df.index)

    # Loop over the intervals
    for interval in intervals:
        # Calculate the rolling max and min log prices for the current interval
        rolling_max = df['Log_Price'].rolling(window=interval).max()
        rolling_min = df['Log_Price'].rolling(window=interval).min()

        print(rolling_max.head())
        print(rolling_min.head())

        # Calculate drawdowns and draw-ups
        drawdowns = rolling_max - df['Log_Price']
        drawups = df['Log_Price'] - rolling_min

        # Store the drawdowns and draw-ups in the DataFrame
        features_df[f'Drawdowns_{interval}min'] = drawdowns
        features_df[f'Drawups_{interval}min'] = drawups

    # Drop the initial rows where the rolling max and min can't be calculated
    features_df.dropna(how='all', inplace=True)

    return features_df

# Example intervals in minutes that might correspond to different intraday time horizons
# intervals_in_minutes = [30, 60, 180, 360, 720]  # 1 minute, 1 hour, 3 hours, 6 hours, 12 hours

# Assuming 'filtered_data' has a datetime index and a 'Trade_Price' column
# Calculate the draw-ups and draw-downs
# draw_features = calculate_drawdowns_drawups(filtered_data, intervals_in_minutes)
# print(draw_features.head())

# Note: This code assumes that 'filtered_data' is already sorted by datetime and has no duplicate indices.


In [ ]:
# draw_features = calculate_drawdowns_drawups(resampled_dataframe, intervals_in_minutes)
draw_features = calculate_drawdowns_drawups(resampled_dataframe, [60])
print(draw_features.head())

# Note: This code assumes that 'filtered_data' is already sorted by datetime and has no duplicate indices.
